In [ ]:
# این کد دو تا دیتا ست که هنوز کامل ران نشده ولی حجم داده هارو کم کردم 
# Adjusted code to be more computer-friendly with reduced data size, epochs, batch size, and model size

# ---------------------------
# 1. Reduce Data Size
# ---------------------------
train_df_sampled = train_df.sample(n=2000, random_state=42)
X_train_mitbih_sampled = train_df_sampled.iloc[:, :186].values
y_train_mitbih_sampled = to_categorical(train_df_sampled[187])

X_test_mitbih_sampled = X_test_mitbih[:2000]
y_test_mitbih_sampled = y_test_mitbih[:2000]

# ---------------------------
# 2. Reduce Batch Size
# ---------------------------
train_mitbih_generator = DataGenerator(X_train_mitbih_sampled, y_train_mitbih_sampled, batch_size=16)

# ---------------------------
# 3. Simplify Model
# ---------------------------
cnn_mitbih_model = Sequential([
    Conv1D(32, kernel_size=3, activation='relu', padding='same', input_shape=(1000, 12), kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),

    Conv1D(64, kernel_size=3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),

    Flatten(),
    Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    Dropout(0.5),

    Dense(y_train_mitbih_sampled.shape[1], activation='softmax')
])

cnn_mitbih_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# ---------------------------
# 4. Train with fewer epochs
# ---------------------------
history_mitbih = cnn_mitbih_model.fit(
    train_mitbih_generator,
    validation_data=(X_test_mitbih_sampled, y_test_mitbih_sampled),
    epochs=5,  # Reduced epochs for quick testing
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

# ---------------------------
# 5. Plot Training Results
# ---------------------------
# Plot MITBIH training accuracy and loss
plt.plot(history_mitbih.history['accuracy'], label='train accuracy')
plt.plot(history_mitbih.history['val_accuracy'], label='val accuracy')
plt.title('MITBIH Accuracy over epochs')
plt.legend()
plt.show()

plt.plot(history_mitbih.history['loss'], label='train loss')
plt.plot(history_mitbih.history['val_loss'], label='val loss')
plt.title('MITBIH Loss over epochs')
plt.legend()
plt.show()


In [ ]:
#  با تگ  pac va pvc این کد دیتا ست دوم با ۸۱ درصد درستی 
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from keras.models import Model
from keras.layers import Dense, Input, Convolution1D, MaxPool1D, Flatten, BatchNormalization, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.regularizers import l2
import itertools

# Load the data from the paths provided
train_df = pd.read_csv("D:/downloads/archive (1)/mitbih_train.csv", header=None)
test_df = pd.read_csv("D:/downloads/archive (1)/mitbih_test.csv", header=None)

# Data balancing using resampling
train_df[187] = train_df[187].astype(int)

# Map the labels to the desired classes (PAC = 1, PVC = 2, Normal = 0)
train_df[187] = train_df[187].apply(lambda x: 1 if x == 1 else (2 if x == 2 else 0))  # 1 -> PAC, 2 -> PVC, 0 -> Normal
test_df[187] = test_df[187].apply(lambda x: 1 if x == 1 else (2 if x == 2 else 0))  # Ensure the same mapping for test data

# Resample the data to handle class imbalance
df_0 = train_df[train_df[187] == 0].sample(n=20000, random_state=42)
df_1 = train_df[train_df[187] == 1]
df_2 = train_df[train_df[187] == 2]

# Upsample classes
df_1_upsample = resample(df_1, replace=True, n_samples=20000, random_state=123)
df_2_upsample = resample(df_2, replace=True, n_samples=20000, random_state=124)

train_df = pd.concat([df_0, df_1_upsample, df_2_upsample])

# Visualizing class distribution
equilibre = train_df[187].value_counts()
plt.figure(figsize=(10, 6))
my_circle = plt.Circle((0, 0), 0.7, color='white')
plt.pie(equilibre, labels=['Normal', 'PAC', 'PVC'], colors=['red', 'green', 'blue'], autopct='%1.1f%%')
p = plt.gcf()
p.gca().add_artist(my_circle)
plt.show()

# Extracting one sample per class for visualization
c = train_df.groupby(187, group_keys=False).apply(lambda train_df: train_df.sample(1))

# Plotting an example from each class
for i in range(3):  # 3 classes: Normal, PAC, PVC
    plt.plot(c.iloc[i, :186])
    plt.title(f"Class {i}")
    plt.show()

# Preprocessing the signals and converting to categorical
X_train = train_df.iloc[:, :186].values
X_test = test_df.iloc[:, :186].values
y_train = to_categorical(train_df[187], num_classes=3)  # 3 classes: Normal, PAC, PVC
y_test = to_categorical(test_df[187], num_classes=3)    # Same for test data

# Adding Gaussian noise to the signals for data augmentation
def add_gaussian_noise(signal):
    noise = np.random.normal(0, 0.5, 186)
    return (signal + noise)

for i in range(len(X_train)):
    X_train[i, :] = add_gaussian_noise(X_train[i, :])

# Reshaping data for CNN
X_train = X_train.reshape(len(X_train), X_train.shape[1], 1)
X_test = X_test.reshape(len(X_test), X_test.shape[1], 1)

# CNN Model with Dropout, L2 Regularization, and Learning Rate Reduction
def create_model(X_train, y_train):
    im_shape = (X_train.shape[1], 1)
    inputs_cnn = Input(shape=(im_shape), name='inputs_cnn')

    # First convolutional block
    conv1_1 = Convolution1D(64, 6, activation='relu', kernel_regularizer=l2(0.01))(inputs_cnn)
    conv1_1 = BatchNormalization()(conv1_1)
    pool1 = MaxPool1D(pool_size=3, strides=2, padding="same")(conv1_1)
    dropout1 = Dropout(0.5)(pool1)

    # Second convolutional block
    conv2_1 = Convolution1D(64, 3, activation='relu', kernel_regularizer=l2(0.01))(dropout1)
    conv2_1 = BatchNormalization()(conv2_1)
    pool2 = MaxPool1D(pool_size=2, strides=2, padding="same")(conv2_1)
    dropout2 = Dropout(0.5)(pool2)

    # Third convolutional block
    conv3_1 = Convolution1D(64, 3, activation='relu', kernel_regularizer=l2(0.01))(dropout2)
    conv3_1 = BatchNormalization()(conv3_1)
    pool3 = MaxPool1D(pool_size=2, strides=2, padding="same")(conv3_1)

    # Fully connected layers
    flatten = Flatten()(pool3)
    dense_end1 = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(flatten)
    dense_end2 = Dense(32, activation='relu', kernel_regularizer=l2(0.01))(dense_end1)
    dropout3 = Dropout(0.5)(dense_end2)

    # Output layer
    main_output = Dense(3, activation='softmax', name='main_output')(dropout3)

    model = Model(inputs=inputs_cnn, outputs=main_output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = create_model(X_train, y_train)

# Callbacks with EarlyStopping and ReduceLROnPlateau
callbacks = [
    EarlyStopping(monitor='val_loss', patience=8),
    ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
]

# Training the model
history = model.fit(X_train, y_train, epochs=40, batch_size=32, validation_data=(X_test, y_test), callbacks=callbacks)
model.load_weights('best_model.h5')

# Evaluate the model
def evaluate_model(history, X_test, y_test, model):
    scores = model.evaluate(X_test, y_test, verbose=0)
    print(f"Accuracy: {scores[1] * 100:.2f}%")
    
    # Plot accuracy
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Model Accuracy')
    plt.show()

    # Plot loss
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Model Loss')
    plt.show()

    # Confusion Matrix
    y_pred = model.predict(X_test)
    y_true = np.argmax(y_test, axis=1)
    y_pred_classes = np.argmax(y_pred, axis=1)

    cm = confusion_matrix(y_true, y_pred_classes)
    plot_confusion_matrix(cm, classes=['Normal', 'PAC', 'PVC'], normalize=True, title="Confusion Matrix")

# Function to plot confusion matrix
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

# Evaluating the model
evaluate_model(history, X_test, y_test, model)


In [ ]:
#این کد داده ست دوم با تگ های خودش که درستی ۹۷ درصد ولی موقع تشخیص خطا یکم اورفیت میشه 
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from tensorflow.keras.utils import to_categorical
from sklearn.utils import class_weight
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import resample
import warnings
from tensorflow.keras.layers import Dense, Convolution1D, MaxPool1D, Flatten, Dropout, BatchNormalization, Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

warnings.filterwarnings('ignore')

# Load the data from the paths provided
train_df = pd.read_csv("D:/downloads/archive (1)/mitbih_train.csv", header=None)
test_df = pd.read_csv("D:/downloads/archive (1)/mitbih_test.csv", header=None)

train_df[187] = train_df[187].astype(int)
equilibre = train_df[187].value_counts()
print(equilibre)
plt.figure(figsize=(20,10))
my_circle = plt.Circle((0, 0), 0.7, color='white')
plt.pie(equilibre, labels=['n', 'q', 'v', 's', 'f'], colors=['red', 'green', 'blue', 'skyblue', 'orange'], autopct='%1.1f%%')
p = plt.gcf()
p.gca().add_artist(my_circle)
plt.show()

df_1 = train_df[train_df[187] == 1]
df_2 = train_df[train_df[187] == 2]
df_3 = train_df[train_df[187] == 3]
df_4 = train_df[train_df[187] == 4]
df_0 = (train_df[train_df[187] == 0]).sample(n=20000, random_state=42)

df_1_upsample = resample(df_1, replace=True, n_samples=20000, random_state=123)
df_2_upsample = resample(df_2, replace=True, n_samples=20000, random_state=124)
df_3_upsample = resample(df_3, replace=True, n_samples=20000, random_state=125)
df_4_upsample = resample(df_4, replace=True, n_samples=20000, random_state=126)

train_df = pd.concat([df_0, df_1_upsample, df_2_upsample, df_3_upsample, df_4_upsample])
equilibre = train_df[187].value_counts()
print(equilibre)
plt.figure(figsize=(20,10))
my_circle = plt.Circle((0, 0), 0.7, color='white')
plt.pie(equilibre, labels=['n', 'q', 'v', 's', 'f'], colors=['red', 'green', 'blue', 'skyblue', 'orange'], autopct='%1.1f%%')
p = plt.gcf()
p.gca().add_artist(my_circle)
plt.show()

# Preprocessing
target_train = train_df[187]
target_test = test_df[187]
y_train = to_categorical(target_train)
y_test = to_categorical(target_test)

X_train = train_df.iloc[:, :186].values
X_test = test_df.iloc[:, :186].values

# Normalize the data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[1]))
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[1]))

# Reshape for CNN
X_train = X_train.reshape(len(X_train), X_train.shape[1], 1)
X_test = X_test.reshape(len(X_test), X_test.shape[1], 1)

def network(X_train, y_train, X_test, y_test):
    im_shape = (X_train.shape[1], 1)
    inputs_cnn = Input(shape=(im_shape), name='inputs_cnn')
    conv1_1 = Convolution1D(64, (6), activation='relu', input_shape=im_shape)(inputs_cnn)
    conv1_1 = BatchNormalization()(conv1_1)
    pool1 = MaxPool1D(pool_size=(3), strides=(2), padding="same")(conv1_1)
    
    conv2_1 = Convolution1D(64, (3), activation='relu')(pool1)
    conv2_1 = BatchNormalization()(conv2_1)
    pool2 = MaxPool1D(pool_size=(2), strides=(2), padding="same")(conv2_1)
    
    conv3_1 = Convolution1D(64, (3), activation='relu')(pool2)
    conv3_1 = BatchNormalization()(conv3_1)
    pool3 = MaxPool1D(pool_size=(2), strides=(2), padding="same")(conv3_1)

    flatten = Flatten()(pool3)
    dense_end1 = Dense(64, activation='relu')(flatten)
    dense_end2 = Dense(32, activation='relu')(dense_end1)
    
    # Add Dropout layers
    dropout1 = Dropout(0.3)(dense_end2)  # Add dropout to prevent overfitting
    
    main_output = Dense(5, activation='softmax', name='main_output')(dropout1)

    model = Model(inputs=inputs_cnn, outputs=main_output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Callbacks
    callbacks = [EarlyStopping(monitor='val_loss', patience=8),
                 ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

    # Fit model
    history = model.fit(X_train, y_train, epochs=40, callbacks=callbacks, batch_size=32, validation_data=(X_test, y_test))

    model.load_weights('best_model.h5')
    return model, history, callbacks

# Ensure target_train contains the original class labels (not one-hot encoded)
target_train = train_df[187].values  # Convert to numpy array

# Calculate class weights for imbalanced data
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(target_train), y=target_train)

# Initialize model and callbacks
model, history, callbacks = network(X_train, y_train, X_test, y_test)

# Use class weights during training
history = model.fit(X_train, y_train, epochs=40, callbacks=callbacks, batch_size=32, validation_data=(X_test, y_test), class_weight=dict(enumerate(class_weights)))

def evaluate_model(history, X_test, y_test, model):
    scores = model.evaluate((X_test), y_test, verbose=0)
    print("Accuracy: %.2f%%" % (scores[1] * 100))

    print(history)
    fig1, ax_acc = plt.subplots()
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Model - Accuracy')
    plt.legend(['Training', 'Validation'], loc='lower right')
    plt.show()

    fig2, ax_loss = plt.subplots()
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Model- Loss')
    plt.legend(['Training', 'Validation'], loc='upper right')
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.show()

    target_names = ['0', '1', '2', '3', '4']

    y_true = []
    for element in y_test:
        y_true.append(np.argmax(element))
    prediction_proba = model.predict(X_test)
    prediction = np.argmax(prediction_proba, axis=1)
    cnf_matrix = confusion_matrix(y_true, prediction)

model, history, callbacks = network(X_train, y_train, X_test, y_test)

evaluate_model(history, X_test, y_test, model)

y_pred = model.predict(X_test)

import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure(figsize=(10, 10))
plot_confusion_matrix(cnf_matrix, classes=['N', 'S', 'V', 'F', 'Q'], normalize=True,
                      title='Confusion matrix, with normalization')
plt.show()


In [ ]:
#این کد دیتا ست دوم بدون اورفیت و ۶۸ درصد درستی با تگ های خودش
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from keras.models import Model
from keras.layers import Dense, Input, Convolution1D, MaxPool1D, Flatten, BatchNormalization, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.regularizers import l2  # Importing l2 regularizer
import itertools

# Load the data from the paths provided
train_df = pd.read_csv("D:/downloads/archive (1)/mitbih_train.csv", header=None)
test_df = pd.read_csv("D:/downloads/archive (1)/mitbih_test.csv", header=None)

# Data balancing using resampling
train_df[187] = train_df[187].astype(int)
df_1 = train_df[train_df[187] == 1]
df_2 = train_df[train_df[187] == 2]
df_3 = train_df[train_df[187] == 3]
df_4 = train_df[train_df[187] == 4]
df_0 = (train_df[train_df[187] == 0]).sample(n=20000, random_state=42)

df_1_upsample = resample(df_1, replace=True, n_samples=20000, random_state=123)
df_2_upsample = resample(df_2, replace=True, n_samples=20000, random_state=124)
df_3_upsample = resample(df_3, replace=True, n_samples=20000, random_state=125)
df_4_upsample = resample(df_4, replace=True, n_samples=20000, random_state=126)

# Concatenating resampled data
train_df = pd.concat([df_0, df_1_upsample, df_2_upsample, df_3_upsample, df_4_upsample])

# Visualizing class distribution
equilibre = train_df[187].value_counts()
plt.figure(figsize=(10, 6))
my_circle = plt.Circle((0, 0), 0.7, color='white')
plt.pie(equilibre, labels=['N', 'S', 'V', 'F', 'Q'], colors=['red', 'green', 'blue', 'skyblue', 'orange'], autopct='%1.1f%%')
p = plt.gcf()
p.gca().add_artist(my_circle)
plt.show()

# Extracting one sample per class for visualization
c = train_df.groupby(187, group_keys=False).apply(lambda train_df: train_df.sample(1))

# Plotting an example from each class
for i in range(5):
    plt.plot(c.iloc[i, :186])
    plt.title(f"Class {i}")
    plt.show()

# Preprocessing the signals and converting to categorical
X_train = train_df.iloc[:, :186].values
X_test = test_df.iloc[:, :186].values
y_train = to_categorical(train_df[187])
y_test = to_categorical(test_df[187])

# Adding Gaussian noise to the signals for data augmentation
def add_gaussian_noise(signal):
    noise = np.random.normal(0, 0.5, 186)
    return (signal + noise)

for i in range(len(X_train)):
    X_train[i, :] = add_gaussian_noise(X_train[i, :])

# Reshaping data for CNN
X_train = X_train.reshape(len(X_train), X_train.shape[1], 1)
X_test = X_test.reshape(len(X_test), X_test.shape[1], 1)

# CNN Model with Dropout and L2 Regularization
def create_model(X_train, y_train):
    im_shape = (X_train.shape[1], 1)
    inputs_cnn = Input(shape=(im_shape), name='inputs_cnn')
    
    conv1_1 = Convolution1D(64, 6, activation='relu', kernel_regularizer=l2(0.01))(inputs_cnn)
    conv1_1 = BatchNormalization()(conv1_1)
    pool1 = MaxPool1D(pool_size=3, strides=2, padding="same")(conv1_1)
    dropout1 = Dropout(0.5)(pool1)
    
    conv2_1 = Convolution1D(64, 3, activation='relu', kernel_regularizer=l2(0.01))(dropout1)
    conv2_1 = BatchNormalization()(conv2_1)
    pool2 = MaxPool1D(pool_size=2, strides=2, padding="same")(conv2_1)
    dropout2 = Dropout(0.5)(pool2)
    
    conv3_1 = Convolution1D(64, 3, activation='relu', kernel_regularizer=l2(0.01))(dropout2)
    conv3_1 = BatchNormalization()(conv3_1)
    pool3 = MaxPool1D(pool_size=2, strides=2, padding="same")(conv3_1)
    
    flatten = Flatten()(pool3)
    dense_end1 = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(flatten)
    dense_end2 = Dense(32, activation='relu', kernel_regularizer=l2(0.01))(dense_end1)
    dropout3 = Dropout(0.5)(dense_end2)
    
    main_output = Dense(5, activation='softmax', name='main_output')(dropout3)

    model = Model(inputs=inputs_cnn, outputs=main_output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = create_model(X_train, y_train)

# Callbacks with ReduceLROnPlateau
callbacks = [
    EarlyStopping(monitor='val_loss', patience=8),
    ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
]

# Training the model
history = model.fit(X_train, y_train, epochs=40, batch_size=32, validation_data=(X_test, y_test), callbacks=callbacks)
model.load_weights('best_model.h5')

# Evaluate the model
def evaluate_model(history, X_test, y_test, model):
    scores = model.evaluate(X_test, y_test, verbose=0)
    print(f"Accuracy: {scores[1] * 100:.2f}%")
    
    # Plot accuracy
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Model Accuracy')
    plt.show()

    # Plot loss
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Model Loss')
    plt.show()

    # Confusion Matrix
    y_pred = model.predict(X_test)
    y_true = np.argmax(y_test, axis=1)
    y_pred_classes = np.argmax(y_pred, axis=1)

    cm = confusion_matrix(y_true, y_pred_classes)
    plot_confusion_matrix(cm, classes=['N', 'S', 'V', 'F', 'Q'], normalize=True, title="Confusion Matrix")

# Function to plot confusion matrix
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

# Evaluating the model
evaluate_model(history, X_test, y_test, model)


In [1]:
#این کد دو تا دیتا ست بدون کم کردن دیتا ها
import ast
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import resample
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import wfdb
import ast

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# ---------------------------
# 1. Load and Preprocess MITBIH Data
# ---------------------------
train_df = pd.read_csv("D:/downloads/archive (1)/mitbih_train.csv", header=None)
test_df = pd.read_csv("D:/downloads/archive (1)/mitbih_test.csv", header=None)

train_df[187] = train_df[187].astype(int)

# Upsample classes to balance dataset
df_1 = train_df[train_df[187] == 1]
df_2 = train_df[train_df[187] == 2]
df_3 = train_df[train_df[187] == 3]
df_4 = train_df[train_df[187] == 4]
df_0 = (train_df[train_df[187] == 0]).sample(n=20000, random_state=42)

df_1_upsample = resample(df_1, replace=True, n_samples=20000, random_state=123)
df_2_upsample = resample(df_2, replace=True, n_samples=20000, random_state=124)
df_3_upsample = resample(df_3, replace=True, n_samples=20000, random_state=125)
df_4_upsample = resample(df_4, replace=True, n_samples=20000, random_state=126)

train_df = pd.concat([df_0, df_1_upsample, df_2_upsample, df_3_upsample, df_4_upsample])

# Preprocessing for MITBIH Data
X_train_mitbih = train_df.iloc[:, :186].values
X_test_mitbih = test_df.iloc[:, :186].values
y_train_mitbih = to_categorical(train_df[187])
y_test_mitbih = to_categorical(test_df[187])

# Normalize the data
scaler = MinMaxScaler()
X_train_mitbih = scaler.fit_transform(X_train_mitbih)
X_test_mitbih = scaler.transform(X_test_mitbih)

# Reshaping for CNN (make it 1000 time steps)
X_train_mitbih = np.pad(X_train_mitbih, ((0, 0), (0, 1000 - X_train_mitbih.shape[1])), 'constant')
X_test_mitbih = np.pad(X_test_mitbih, ((0, 0), (0, 1000 - X_test_mitbih.shape[1])), 'constant')

# Make the MITBIH data have 12 channels by repeating the same data across 12 channels
X_train_mitbih = np.repeat(X_train_mitbih[:, :, np.newaxis], 12, axis=2)
X_test_mitbih = np.repeat(X_test_mitbih[:, :, np.newaxis], 12, axis=2)

# ---------------------------
# 2. Load and Preprocess PTB-XL Data
# ---------------------------
path_to_csv = r"E:\ptbxl_database.csv"  # Updated path
base_path = r"D:\downloads\ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3\ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3"

df = pd.read_csv(path_to_csv)
df['scp_codes'] = df['scp_codes'].apply(lambda x: ast.literal_eval(x))

# Map SCP codes to superclass labels
scp_to_superclass = {
    'NORM': 'NORM', 'SR': 'NORM',
    'IMI': 'MI', 'AMI': 'MI', 'ASMI': 'MI',
    'NST_': 'STTC', 'NDT': 'STTC',
    'RBBB': 'CD', 'LBBB': 'CD', 'IVCD': 'CD',
    'LVH': 'HYP', 'HYP': 'HYP'
}

def derive_superclass(row):
    for code in row['scp_codes'].keys():
        if code in scp_to_superclass:
            return scp_to_superclass[code]
    return None

df['superclass'] = df.apply(derive_superclass, axis=1)
filtered_df = df[df['superclass'].notnull()].copy()

superclass_map = {'NORM': 0, 'MI': 1, 'STTC': 2, 'CD': 3, 'HYP': 4}
filtered_df['label'] = filtered_df['superclass'].map(superclass_map)

# Check if filtered_df is empty
print(f"Filtered dataframe shape: {filtered_df.shape}")
if filtered_df.shape[0] == 0:
    print("No records in filtered_df.")
else:
    print("Filtered dataframe has records.")

# ---------------------------
# 3. Load Signals Function with Debugging
# ---------------------------
def load_signals(df, max_length=1000):
    data, labels = [], []
    for _, row in df.iterrows():
        record_path = os.path.join(base_path, row['filename_lr'])
        
        if not (os.path.isfile(record_path + ".hea") and os.path.isfile(record_path + ".dat")):
            continue

        try:
            signal, _ = wfdb.rdsamp(record_path)
        except:
            continue

        print(f"Processing {row['filename_lr']}...")  # Debugging line to check which records are being loaded
        signal = tf.keras.preprocessing.sequence.pad_sequences(
            signal.T, maxlen=max_length, padding='post', truncating='post', dtype='float32'
        ).T
        
        data.append(signal)
        labels.append(row['label'])

    print(f"Loaded {len(data)} signals and labels.")  # Check how many valid signals and labels were loaded
    return np.array(data), np.array(labels)

X_ptb, y_ptb = load_signals(filtered_df, max_length=1000)

# ---------------------------
# 4. One-hot encode PTB labels
# ---------------------------
y_ptb_encoded = to_categorical(y_ptb)

# ---------------------------
# 5. Use Data Generators to Avoid Memory Overload
# ---------------------------
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, X_data, y_data, batch_size=32, shuffle=True):
        self.X_data = X_data
        self.y_data = y_data
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indices = np.arange(len(X_data))
        
    def __len__(self):
        return int(np.floor(len(self.X_data) / self.batch_size))
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)
    
    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch_X = self.X_data[batch_indices]
        batch_y = self.y_data[batch_indices]
        return batch_X, batch_y

train_mitbih_generator = DataGenerator(X_train_mitbih, y_train_mitbih, batch_size=32)
train_ptb_generator = DataGenerator(X_ptb, y_ptb_encoded, batch_size=32)

# ---------------------------
# 6. CNN Model for MITBIH with Increased Regularization
# ---------------------------
cnn_mitbih_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', padding='same', input_shape=(1000, 12), kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),  # Increased dropout
    
    Conv1D(128, kernel_size=5, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),  # Increased dropout
    
    Flatten(),
    Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    Dropout(0.5),  # Increased dropout
    
    Dense(y_train_mitbih.shape[1], activation='softmax')
])

cnn_mitbih_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# ---------------------------
# 7. Train the MITBIH model
# ---------------------------
early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001)

history_mitbih = cnn_mitbih_model.fit(
    train_mitbih_generator,
    validation_data=(X_test_mitbih, y_test_mitbih),  # Add validation data
    epochs=50,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

# ---------------------------
# 8. Save the MITBIH model
# ---------------------------
cnn_mitbih_model.save('mitbih_cnn_model.keras', save_format='keras')

# ---------------------------
# 9. CNN Model for PTB-XL with Increased Regularization
# ---------------------------
cnn_ptb_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', padding='same', input_shape=(1000, 12), kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),  # Increased dropout
    
    Conv1D(128, kernel_size=5, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),  # Increased dropout
    
    Flatten(),
    Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    Dropout(0.5),  # Increased dropout
    
    Dense(y_ptb_encoded.shape[1], activation='softmax')
])

cnn_ptb_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# ---------------------------
# 10. Train the PTB-XL model
# ---------------------------
history_ptb = cnn_ptb_model.fit(
    train_ptb_generator,
    validation_data=(X_test, y_test_ptb_encoded),  # Add validation data
    epochs=50,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

# ---------------------------
# 11. Save the PTB-XL model
# ---------------------------
cnn_ptb_model.save('ptbxl_cnn_model.keras', save_format='keras')

# ---------------------------
# 12. Plot Training and Validation Accuracy and Loss
# ---------------------------

# MITBIH accuracy and loss
plt.plot(history_mitbih.history['accuracy'], label='train accuracy')
plt.plot(history_mitbih.history['val_accuracy'], label='val accuracy')
plt.title('MITBIH Accuracy over epochs')
plt.legend()
plt.show()

plt.plot(history_mitbih.history['loss'], label='train loss')
plt.plot(history_mitbih.history['val_loss'], label='val loss')
plt.title('MITBIH Loss over epochs')
plt.legend()
plt.show()

# PTB-XL accuracy and loss
plt.plot(history_ptb.history['accuracy'], label='train accuracy')
plt.plot(history_ptb.history['val_accuracy'], label='val accuracy')
plt.title('PTB-XL Accuracy over epochs')
plt.legend()
plt.show()

plt.plot(history_ptb.history['loss'], label='train loss')
plt.plot(history_ptb.history['val_loss'], label='val loss')
plt.title('PTB-XL Loss over epochs')
plt.legend()
plt.show()


KeyboardInterrupt: 

In [ ]:
#این کد دیتا ست اول با ۹۵ درصد درسنی 
import pandas as pd  
import numpy as np  
import wfdb  
import os  
from tensorflow.keras.models import Sequential  
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense  
from tensorflow.keras.utils import to_categorical  
from sklearn.model_selection import train_test_split  

# Set the path to your dataset  
path = r'C:\Users\Asus\Downloads\ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3\ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3'  

# Load the main data CSV file  
metadata = pd.read_csv(os.path.join(path, 'ptbxl_database.csv'))  

# Define a function to load the signal data  
def load_signals(df):  
    data = []  
    labels = []  
    
    for index, row in df.iterrows():  
        # Get the base filename and adjust as needed  
        base_filename = row['filename_lr']  
        
        # Folder path remains static since all files appear in '00000'  
        record_folder = os.path.join(path)  
        
        # Construct the complete path for .hea and .dat files  
        record_path = os.path.join(record_folder, base_filename)  # Points to the .hea and .dat files  
        
        print(f"Loading: {record_path}")  # Debug statement  
        
        try:  
            # Check for required files and load them  
            if os.path.isfile(f"{record_path}.hea") and os.path.isfile(f"{record_path}.dat"):  
                signal, _ = wfdb.rdsamp(record_path)  # Read the signal  
                
                # Append to data and labels based on scp_codes  
                if 'PAC' in row['scp_codes']:  
                    data.append(signal)  
                    labels.append(1)  # Label for PAC  
                elif 'NORM' in row['scp_codes']:  
                    data.append(signal)  
                    labels.append(0)  # Label for Normal  
            else:  
                print(f"Required files not found for {record_path}")  
        except Exception as e:  
            print(f"Error loading {record_path}: {e}")  
        
    return np.array(data), np.array(labels)  

# Load the data  
X, y = load_signals(metadata)  

# Check if any data was loaded  
if X.size == 0:  
    raise ValueError("No signal data was loaded; please check your file paths and structure.")  

# Convert labels to categorical (one-hot encoding)  
y = to_categorical(y)  

# Split the dataset into training and testing sets (80-20 split)  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  

# Build the CNN Model  
model = Sequential()  
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X.shape[1], X.shape[2])))  # Adjust input_shape to match number of channels  
model.add(MaxPooling1D(pool_size=2))  
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))  
model.add(MaxPooling1D(pool_size=2))  
model.add(Flatten())  
model.add(Dense(64, activation='relu'))  
model.add(Dense(y.shape[1], activation='softmax'))  # Number of classes  

# Compile the model  
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  

# Train the model  
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)  

# Evaluate the model  
loss, accuracy = model.evaluate(X_test, y_test)  
print(f'Test Loss: {loss}')  
print(f'Test Accuracy: {accuracy}')  

# Save the model if needed  
model.save('pac_detection_model.h5')

In [ ]:
#این کد دیتا ست اول با راستی ازماییی دیتا ها با درستی ۷۷ درصد 
import os
import ast
import wfdb
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# ---------------------------
# 1. بارگذاری دیتاست
# ---------------------------
path_to_csv = r"C:\Users\Asus\Desktop\ptbxl_database.csv"

base_path = r"D:\downloads\ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3\ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3"

df = pd.read_csv(path_to_csv)
df['scp_codes'] = df['scp_codes'].apply(lambda x: ast.literal_eval(x))

# ---------------------------
# 2. نگاشت کلاس‌ها
# ---------------------------
scp_to_superclass = {
    'NORM': 'NORM', 'SR': 'NORM',
    'IMI': 'MI', 'AMI': 'MI', 'ASMI': 'MI',
    'NST_': 'STTC', 'NDT': 'STTC',
    'RBBB': 'CD', 'LBBB': 'CD', 'IVCD': 'CD',
    'LVH': 'HYP', 'HYP': 'HYP'
}

def derive_superclass(row):
    for code in row['scp_codes'].keys():
        if code in scp_to_superclass:
            return scp_to_superclass[code]
    return None

df['superclass'] = df.apply(derive_superclass, axis=1)
filtered_df = df[df['superclass'].notnull()].copy()

superclass_map = {'NORM': 0, 'MI': 1, 'STTC': 2, 'CD': 3, 'HYP': 4}
filtered_df['label'] = filtered_df['superclass'].map(superclass_map)

# ---------------------------
# 3. بارگذاری سیگنال‌های ECG
# ---------------------------
def load_signals(df, max_length=1000):
    data, labels, ids = [], [], []
    for _, row in df.iterrows():
        record_path = os.path.join(base_path, row['filename_lr'])

        if not (os.path.isfile(record_path + ".hea") and os.path.isfile(record_path + ".dat")):
            continue

        try:
            signal, _ = wfdb.rdsamp(record_path)
        except:
            continue

        signal = tf.keras.preprocessing.sequence.pad_sequences(
            signal.T, maxlen=max_length, padding='post', truncating='post', dtype='float32'
        ).T
        
        data.append(signal)
        labels.append(row['label'])
        ids.append(row['filename_lr'])

    return np.array(data), np.array(labels), ids

X, y, ids = load_signals(filtered_df, max_length=1000)

# ---------------------------
# 4. تقسیم داده‌ها به مجموعه آموزش و آزمون
# ---------------------------
try:
    X_train, X_test, y_train, y_test, ids_train, ids_test = train_test_split(
        X, y, ids, test_size=0.2, random_state=42, stratify=y
    )
except ValueError:
    X_train, X_test, y_train, y_test, ids_train, ids_test = train_test_split(
        X, y, ids, test_size=0.2, random_state=42
    )

# ---------------------------
# 5. پردازش اولیه: نرمال‌سازی داده‌ها
# ---------------------------
scaler = StandardScaler()
X_train = X_train.reshape(X_train.shape[0], -1)
X_train = scaler.fit_transform(X_train)
X_train = X_train.reshape(-1, 1000, X.shape[2])

X_test = X_test.reshape(X_test.shape[0], -1)
X_test = scaler.transform(X_test)
X_test = X_test.reshape(-1, 1000, X.shape[2])

# برچسب‌ها به قالب one-hot
y_train_cat = to_categorical(y_train)
y_test_cat  = to_categorical(y_test)

# ---------------------------
# 6. مدل CNN-LSTM
# ---------------------------
model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', padding='same', input_shape=(1000, X.shape[2])),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    Conv1D(128, kernel_size=5, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    Conv1D(256, kernel_size=3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    LSTM(100, return_sequences=True),
    LSTM(50),
    
    Dense(128, activation='relu'),
    Dropout(0.3),
    
    Dense(y_train_cat.shape[1], activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

# ---------------------------
# 7. آموزش مدل
# ---------------------------
history = model.fit(
    X_train, y_train_cat,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

# ---------------------------
# 8. ارزیابی مدل
# ---------------------------
loss, acc = model.evaluate(X_test, y_test_cat, verbose=1)
print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {acc:.4f}")

# ---------------------------
# 9. پیش‌بینی و نمایش نمونه‌ها
# ---------------------------
preds = model.predict(X_test)
rev_map = {v: k for k, v in superclass_map.items()}

rand_idx = np.random.choice(len(X_test), 5, replace=False)
for idx in rand_idx:
    true_label = rev_map[y_test[idx]]
    pred_label = rev_map[np.argmax(preds[idx])]
    print(f"ID: {ids_test[idx]}, Predicted: {pred_label}, True: {true_label}")

# ذخیره مدل
model.save('my_ecg_model.keras', save_format='keras')

In [ ]:
#این کد دیتا ست اول با راستی ازماییی دیتا ها با درستی ۷۸ درصد
import os
import ast
import wfdb
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, LSTM, GlobalAveragePooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# ---------------------------
# 1. بارگذاری دیتاست
# ---------------------------
path_to_csv = r"E:\ptbxl_database.csv"
base_path   = r"D:\downloads\ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3\ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3"

df = pd.read_csv(path_to_csv)
df['scp_codes'] = df['scp_codes'].apply(lambda x: ast.literal_eval(x))

# ---------------------------
# 2. نگاشت کلاس‌ها
# ---------------------------
scp_to_superclass = {
    'NORM': 'NORM', 'SR': 'NORM',
    'IMI': 'MI', 'AMI': 'MI', 'ASMI': 'MI',
    'NST_': 'STTC', 'NDT': 'STTC',
    'RBBB': 'CD', 'LBBB': 'CD', 'IVCD': 'CD',
    'LVH': 'HYP', 'HYP': 'HYP'
}

def derive_superclass(row):
    for code in row['scp_codes'].keys():
        if code in scp_to_superclass:
            return scp_to_superclass[code]
    return None

df['superclass'] = df.apply(derive_superclass, axis=1)
filtered_df = df[df['superclass'].notnull()].copy()

superclass_map = {'NORM': 0, 'MI': 1, 'STTC': 2, 'CD': 3, 'HYP': 4}
filtered_df['label'] = filtered_df['superclass'].map(superclass_map)

# ---------------------------
# 3. بارگذاری سیگنال‌های ECG
# ---------------------------
def load_signals(df, max_length=1000):
    data, labels, ids = [], [], []
    for _, row in df.iterrows():
        record_path = os.path.join(base_path, row['filename_lr'])

        if not (os.path.isfile(record_path + ".hea") and os.path.isfile(record_path + ".dat")):
            continue

        try:
            signal, _ = wfdb.rdsamp(record_path)
        except:
            continue

        signal = tf.keras.preprocessing.sequence.pad_sequences(
            signal.T, maxlen=max_length, padding='post', truncating='post', dtype='float32'
        ).T
        
        data.append(signal)
        labels.append(row['label'])
        ids.append(row['filename_lr'])

    return np.array(data), np.array(labels), ids

X, y, ids = load_signals(filtered_df, max_length=1000)

# ---------------------------
# 4. تقسیم داده‌ها به مجموعه آموزش و آزمون
# ---------------------------
try:
    X_train, X_test, y_train, y_test, ids_train, ids_test = train_test_split(
        X, y, ids, test_size=0.2, random_state=42, stratify=y
    )
except ValueError:
    X_train, X_test, y_train, y_test, ids_train, ids_test = train_test_split(
        X, y, ids, test_size=0.2, random_state=42
    )

# ---------------------------
# 5. پردازش اولیه: نرمال‌سازی داده‌ها
# ---------------------------
scaler = StandardScaler()
X_train = X_train.reshape(X_train.shape[0], -1)
X_train = scaler.fit_transform(X_train)
X_train = X_train.reshape(-1, 1000, X.shape[2])

X_test = X_test.reshape(X_test.shape[0], -1)
X_test = scaler.transform(X_test)
X_test = X_test.reshape(-1, 1000, X.shape[2])

# برچسب‌ها به قالب one-hot
y_train_cat = to_categorical(y_train)
y_test_cat  = to_categorical(y_test)

# ---------------------------
# 6. مدل CNN-LSTM
# ---------------------------
model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', padding='same', input_shape=(1000, X.shape[2])),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    Conv1D(128, kernel_size=3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    Conv1D(256, kernel_size=3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    GlobalAveragePooling1D(),

    Dense(128, activation='relu'),
    Dropout(0.5),

    Dense(y_train_cat.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Learning rate scheduler to reduce learning rate when validation loss plateaus
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

# ---------------------------
# 7. آموزش مدل
# ---------------------------
history = model.fit(
    X_train, y_train_cat,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop, lr_scheduler],
    verbose=1
)

# ---------------------------
# 8. ارزیابی مدل
# ---------------------------
loss, acc = model.evaluate(X_test, y_test_cat, verbose=1)
print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {acc:.4f}")

# ---------------------------
# 9. پیش‌بینی و نمایش نمونه‌ها
# ---------------------------
preds = model.predict(X_test)
rev_map = {v: k for k, v in superclass_map.items()}

rand_idx = np.random.choice(len(X_test), 5, replace=False)
for idx in rand_idx:
    true_label = rev_map[y_test[idx]]
    pred_label = rev_map[np.argmax(preds[idx])]
    print(f"ID: {ids_test[idx]}, Predicted: {pred_label}, True: {true_label}")

# ذخیره مدل
model.save('my_ecg_model_improved.keras', save_format='keras')


In [ ]:
#این کد دو تا دیتا ست ولی جدا از هم 
import os
import ast
import wfdb
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, GlobalAveragePooling1D, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt  # plotting
import numpy as np  # linear algebra

# ---------------------------
# 1. Load Dataset
# ---------------------------
path_to_csv = r"E:\ptbxl_database.csv"
base_path = r"D:\downloads\ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3\ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3"

df = pd.read_csv(path_to_csv)
df['scp_codes'] = df['scp_codes'].apply(lambda x: ast.literal_eval(x))

# ---------------------------
# 2. Map Superclass Labels
# ---------------------------
scp_to_superclass = {
    'NORM': 'NORM', 'SR': 'NORM',
    'IMI': 'MI', 'AMI': 'MI', 'ASMI': 'MI',
    'NST_': 'STTC', 'NDT': 'STTC',
    'RBBB': 'CD', 'LBBB': 'CD', 'IVCD': 'CD',
    'LVH': 'HYP', 'HYP': 'HYP'
}

def derive_superclass(row):
    for code in row['scp_codes'].keys():
        if code in scp_to_superclass:
            return scp_to_superclass[code]
    return None

df['superclass'] = df.apply(derive_superclass, axis=1)
filtered_df = df[df['superclass'].notnull()].copy()

superclass_map = {'NORM': 0, 'MI': 1, 'STTC': 2, 'CD': 3, 'HYP': 4}
filtered_df['label'] = filtered_df['superclass'].map(superclass_map)

# ---------------------------
# 3. Load ECG Signals
# ---------------------------
def load_signals(df, max_length=1000):
    data, labels, ids = [], [], []
    for _, row in df.iterrows():
        record_path = os.path.join(base_path, row['filename_lr'])

        if not (os.path.isfile(record_path + ".hea") and os.path.isfile(record_path + ".dat")):
            continue

        try:
            signal, _ = wfdb.rdsamp(record_path)
        except:
            continue

        signal = tf.keras.preprocessing.sequence.pad_sequences(
            signal.T, maxlen=max_length, padding='post', truncating='post', dtype='float32'
        ).T
        
        data.append(signal)
        labels.append(row['label'])
        ids.append(row['filename_lr'])

    return np.array(data), np.array(labels), ids

X, y, ids = load_signals(filtered_df, max_length=1000)

# ---------------------------
# 4. Train-Test Split
# ---------------------------
X_train, X_test, y_train, y_test, ids_train, ids_test = train_test_split(
    X, y, ids, test_size=0.2, random_state=42, stratify=y
)

# ---------------------------
# 5. Data Preprocessing: Normalization
# ---------------------------
scaler = StandardScaler()
X_train = X_train.reshape(X_train.shape[0], -1)
X_train = scaler.fit_transform(X_train)
X_train = X_train.reshape(-1, 1000, X.shape[2])

X_test = X_test.reshape(X_test.shape[0], -1)
X_test = scaler.transform(X_test)
X_test = X_test.reshape(-1, 1000, X.shape[2])

# One-hot encoding of labels
y_train_cat = to_categorical(y_train)
y_test_cat  = to_categorical(y_test)

# ---------------------------
# 6. CNN-LSTM Model
# ---------------------------
model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', padding='same', input_shape=(1000, X.shape[2])),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    Conv1D(128, kernel_size=3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    Conv1D(256, kernel_size=3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    GlobalAveragePooling1D(),

    Dense(128, activation='relu'),
    Dropout(0.5),

    Dense(y_train_cat.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Learning rate scheduler to reduce learning rate when validation loss plateaus
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

# ---------------------------
# 7. Train the Model
# ---------------------------
history = model.fit(
    X_train, y_train_cat,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop, lr_scheduler],
    verbose=1
)

# ---------------------------
# 8. Model Evaluation
# ---------------------------
loss, acc = model.evaluate(X_test, y_test_cat, verbose=1)
print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {acc:.4f}")

# ---------------------------
# 9. Prediction and Display Samples
# ---------------------------
preds = model.predict(X_test)
rev_map = {v: k for k, v in superclass_map.items()}

rand_idx = np.random.choice(len(X_test), 5, replace=False)
for idx in rand_idx:
    true_label = rev_map[y_test[idx]]
    pred_label = rev_map[np.argmax(preds[idx])]
    print(f"ID: {ids_test[idx]}, Predicted: {pred_label}, True: {true_label}")

# Save the model
model.save('my_ecg_model_improved.keras', save_format='keras')

# ---------------------------
# Visualize Dataset
# ---------------------------
def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]]  # For displaying purposes, pick columns with between 1 and 50 unique values
    nRow, nCol = df.shape
    columnNames = list(df)
    
    # Ensure nGraphRow is an integer by using np.ceil to round up
    nGraphRow = int(np.ceil(nCol / nGraphPerRow))
    
    plt.figure(num=None, figsize=(6 * nGraphPerRow, 8 * nGraphRow), dpi=80, facecolor='w', edgecolor='k')
    
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = df.iloc[:, i]
        if not np.issubdtype(type(columnDf.iloc[0]), np.number):
            valueCounts = columnDf.value_counts()
            valueCounts.plot.bar()
        else:
            columnDf.hist()
        plt.ylabel('counts')
        plt.xticks(rotation=90)
        plt.title(f'{columnNames[i]} (column {i})')
    
    plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)
    plt.show()

# Loop over files in the directory to load and plot the data
for dirname, _, filenames in os.walk("D:/downloads/archive (1)/"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Load and visualize data for mitbih_test.csv
nRowsRead = 1000  # specify 'None' if want to read whole file
df1 = pd.read_csv("D:/downloads/archive (1)/mitbih_test.csv", delimiter=',', nrows=nRowsRead)
df1.dataframeName = 'mitbih_test.csv'
nRow, nCol = df1.shape
print(f'There are {nRow} rows and {nCol} columns')
df1.head(5)
plotPerColumnDistribution(df1, 10, 5)
plotCorrelationMatrix(df1, 25)
plotScatterMatrix(df1, 20, 10)

# Load and visualize data for mitbih_train.csv
nRowsRead = 1000  # specify 'None' if want to read whole file
df2 = pd.read_csv("D:/downloads/archive (1)/mitbih_train.csv", delimiter=',', nrows=nRowsRead)
df2.dataframeName = 'mitbih_train.csv'
nRow, nCol = df2.shape
print(f'There are {nRow} rows and {nCol} columns')
df2.head(5)
plotPerColumnDistribution(df2, 10, 5)
plotCorrelationMatrix(df2, 25)
plotScatterMatrix(df2, 20, 10)

# Load and visualize data for ptbdb_abnormal.csv
nRowsRead = 1000  # specify 'None' if want to read whole file
df3 = pd.read_csv("D:/downloads/archive (1)/ptbdb_abnormal.csv", delimiter=',', nrows=nRowsRead)
df3.dataframeName = 'ptbdb_abnormal.csv'
nRow, nCol = df3.shape
print(f'There are {nRow} rows and {nCol} columns')
df3.head(5)
plotPerColumnDistribution(df3, 10, 5)
plotCorrelationMatrix(df3, 25)
plotScatterMatrix(df3, 20, 10)

nRowsRead = 1000  # specify 'None' if want to read whole file
df4 = pd.read_csv("D:/downloads/archive (1)/ptbdb_normal.csv", delimiter=',', nrows=nRowsRead)
df4.dataframeName = 'ptbdb_normal.csv'
nRow, nCol = df4.shape
print(f'There are {nRow} rows and {nCol} columns')
df4.head(5)
plotPerColumnDistribution(df4, 10, 5)
plotCorrelationMatrix(df4, 25)
plotScatterMatrix(df4, 20, 10)
